In [1]:
#Задача - научиться отделять спам от неспама
#будем рабоать с базой отсюда: https://www.kaggle.com/uciml/sms-spam-collection-dataset

import pandas as pd
import numpy as np

df = pd.read_csv('spam.csv', encoding = 'windows-1251', names = ['Type', 'Text', 'NaN1', 'NaN2', 'NaN3'])
df = df.drop(['NaN1', 'NaN2', 'NaN3'], axis = 1).drop([0]).reset_index(drop = True)
df.head(15)

#df.query('Type=="spam"')

,Type,Text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


In [2]:
# пердобработка + разделяем на train/test
# в данном случае 0 == ham, 1 == spam

y_all = []
for tmp in df['Type']:
    y_all.append((lambda x: x == 'spam' and 1 or 0)(tmp))

sep_num = int(len(df) * 0.7)

x_train, y_train = list(df['Text'][:sep_num]), list(y_all[:sep_num])
x_test, y_test = list(df['Text'][sep_num:]), list(y_all[sep_num:])

for i in range(5):
    print(x_train[i], ' - ', y_train[i], '\n')

Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...  -  0 

Ok lar... Joking wif u oni...  -  0 

Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's  -  1 

U dun say so early hor... U c already then say...  -  0 

Nah I don't think he goes to usf, he lives around here though  -  0 



# --------------------------------------------------------------------------------------------------------------
## TF-IDF:




In [3]:
# Токенизируем входные данные и строим словарь всех известных слов

from sklearn.svm import SVC 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

vectorizer = TfidfVectorizer()
train_x_vect = vectorizer.fit_transform(x_train)
#print(train_x_vect)

In [4]:
train_x_vect.toarray().shape

#по всем 3900 сообщениям набралось 7252 уникальных слов

(3900, 7252)

In [5]:
#попробуем линейное ядро - выберем наиболее оптимальный параметр перебором

grid = {'C': np.power(10.0, np.arange(-5, 6))}

cv = KFold(n_splits = 5, shuffle = True, random_state = 777)
clf = SVC(kernel = 'linear', random_state = 777)

gs = GridSearchCV(clf, grid, scoring = 'accuracy', cv = cv)
gs.fit(train_x_vect, y_train)

print(gs.best_score_)
print(gs.best_params_)

0.9828205128205129
{'C': 10.0}


In [6]:
clf = SVC(kernel = 'linear', random_state = 777, C = 1)
clf.fit(train_x_vect, y_train)

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=777,
    shrinking=True, tol=0.001, verbose=False)

In [7]:
clf.coef_.toarray()

array([[ 0.48971179,  0.63768299,  0.        , ..., -0.06368171,
        -0.06368171,  0.16861006]])

In [8]:
A = clf.coef_.indices
B = np.abs(clf.coef_.data)

Words = A[B.argsort()[-20:][::-1]]
for i in Words:
    print(vectorizer.get_feature_names()[i])    #последние - самые важные для моедли слова

txt
claim
service
uk
150p
www
text
http
reply
chat
18
mobile
ac
еј1
call
ringtone
won
prize
88066
urgent


In [9]:
#проверяем на тестовых данных
            
X_test = np.array(x_test)
test_x_pred_vect = vectorizer.transform(X_test)

data_y_pred = clf.predict(test_x_pred_vect)

for i in data_y_pred[-10:]:
    print(i)

0
0
0
0
1
1
0
0
0
0


In [10]:
X_test[-5] #при данном randomstate 0 - спам, 1 - не спам

'This is the 2nd time we have tried 2 contact u. U have won the еЈ750 Pound prize. 2 claim is easy, call 087187272008 NOW1! Only 10p per minute. BT-national-rate.'

In [11]:
#Сравнение разных метрик
#в данном случае negatine = ham, positive = spam


test_len = len(y_test)
pos_num = sum(y_test)
neg_num = test_len - pos_num

true_pos = 0
true_neg = 0
for i in range(test_len):
    if data_y_pred[i] == y_test[i] == 1:
        true_pos += 1
    if data_y_pred[i] == y_test[i] == 0:
        true_neg += 1

false_pos = pos_num - true_pos
false_neg = neg_num - true_neg

print('accuracy =', (true_pos + true_neg) / test_len)
print('precision = ', true_pos / (true_pos + false_pos))
print('recall = ', true_pos / (true_pos + false_neg))

accuracy = 0.9868421052631579
precision =  0.9166666666666666
recall =  0.9858490566037735


In [12]:
new_df = pd.read_excel('spam_testing.xlsx')
new = []

for str_ in new_df['Spam']:
    new.append(str_)

In [13]:
new_pred_vect = vectorizer.transform(new)
new_pred = clf.predict(new_pred_vect)

for i in range(len(new)):
    print(new[i], ' - ', (lambda x: x == 1 and 'spam' or 'ham')(new_pred[i]), '\n')


We shall await your immediate correspondence with the information requested above to his private email address at (raphaelbostic@usa.com) for fast re-confirmation so that he may conclude your payment immediately.  -  ham 

You are everywhere dirt, on the floor, the windows, even on my shirt. And sometimes when i open my mouth, you are all that comes flowing out. I dream of my world without you, then half my chores are out too. A time of joy for me, lots of tv shows i.ll see. But i guess like all things you just must exist, like rain, hail and mist, and when my time here is done, you and i become one.  -  ham 

We are pleased to inform you that an amount of £500,000.00 (GBP) has been donated and given, gifted to you and your family by Steve & Lenka Thomson, who won the EuroMillions jackpot, lottery of £105,100,701.90 Euro Millions, part of this donation,       it is for you and your family. This donation is to help fight against CoronaVirus COVID-19 pandemic in the world, and help the p

# --------------------------------------------------------------------------------------------------------------
## Триграммы:

In [14]:
#для сокращения объема будем всё приводить к нижнему регистру

#alphabet = list('abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ')
alphabet = list('abcdefghijklmnopqrstuvwxyz.')
numbers_expanded = list('1234567890 ')
trigram_vec = []

for sym_1 in alphabet:
    for sym_2 in alphabet:
        
        trigram_vec.append(' ' + sym_1 + sym_2)
        trigram_vec.append(sym_1 + ' ' + sym_2)
        trigram_vec.append(sym_1 + sym_2 + ' ')
        
        for sym_3 in alphabet:
            trigram_vec.append(sym_1 + sym_2 + sym_3)
            
#for sym_1 in numbers_expanded:
#    for sym_2 in numbers_expanded:
#        trigram_vec.append(sym_1 + sym_2 + '%')
#        trigram_vec.append(sym_1 + sym_2 + '$')

for str_ in ['!!!', '!!?', '!?!', '?!!', '?!?', '??!', '???']:
    trigram_vec.append(str_)

print(trigram_vec[:150], '\n')
print('. . . . . ', '\n')
print(trigram_vec[-150:])

[' aa', 'a a', 'aa ', 'aaa', 'aab', 'aac', 'aad', 'aae', 'aaf', 'aag', 'aah', 'aai', 'aaj', 'aak', 'aal', 'aam', 'aan', 'aao', 'aap', 'aaq', 'aar', 'aas', 'aat', 'aau', 'aav', 'aaw', 'aax', 'aay', 'aaz', 'aa.', ' ab', 'a b', 'ab ', 'aba', 'abb', 'abc', 'abd', 'abe', 'abf', 'abg', 'abh', 'abi', 'abj', 'abk', 'abl', 'abm', 'abn', 'abo', 'abp', 'abq', 'abr', 'abs', 'abt', 'abu', 'abv', 'abw', 'abx', 'aby', 'abz', 'ab.', ' ac', 'a c', 'ac ', 'aca', 'acb', 'acc', 'acd', 'ace', 'acf', 'acg', 'ach', 'aci', 'acj', 'ack', 'acl', 'acm', 'acn', 'aco', 'acp', 'acq', 'acr', 'acs', 'act', 'acu', 'acv', 'acw', 'acx', 'acy', 'acz', 'ac.', ' ad', 'a d', 'ad ', 'ada', 'adb', 'adc', 'add', 'ade', 'adf', 'adg', 'adh', 'adi', 'adj', 'adk', 'adl', 'adm', 'adn', 'ado', 'adp', 'adq', 'adr', 'ads', 'adt', 'adu', 'adv', 'adw', 'adx', 'ady', 'adz', 'ad.', ' ae', 'a e', 'ae ', 'aea', 'aeb', 'aec', 'aed', 'aee', 'aef', 'aeg', 'aeh', 'aei', 'aej', 'aek', 'ael', 'aem', 'aen', 'aeo', 'aep', 'aeq', 'aer', 'aes', 'aet'

In [15]:
#пример того, как c помощью триграмм стоится столбец признаков test_trigram_vec
#test_trigram_vec - сильно разреженый вектор, поэтому вывожу здесь только сами триграммы

test = df.iloc[0]['Text'].lower()
print(test, '\n')

test_trigram_vec = []

for trig in trigram_vec:
    
    test_trigram_vec.append((lambda x: x and 1 or 0)(trig in test))
    
    if trig in test:
        print(trig, end = ', ')
        
#print(test_trigram_vec)

go until jurong point, crazy.. available only in bugis n great world la e buffet... cine there got amore wat... 

abl, a e, ail,  am, amo, at , at.,  av, ava, azy, ble,  bu, buf, bug,  ci, cin,  cr, cra, d l, eat, e b, e g, e o, ere, e t, et., e w, fet, ffe, gis,  go, go , got, g p,  gr, gre, her, il , ila,  in, in , ine, int, is ,  ju, jur,  la, la , lab, ld , le , l j, ly , mor, n b, ne , n g, ng , nly, nti, oin,  on, ong, onl, ore, orl, ot , o u,  po, poi, raz, re , rea, rld, ron, s n, t a,  th, the, til, t w, t.., uff, ugi,  un, unt, uro, vai,  wa, wat,  wo, wor, y i, y.., zy., . a, . c, .. , ..., 

In [16]:
def DataTrigramVec(data):
    
    data.lower()
    data_trigram_vec = []

    for trig in trigram_vec:
        data_trigram_vec.append((lambda x: x and 1 or 0)(trig in data))

    return data_trigram_vec

In [17]:
X_train = []
for tmp in x_train:
    X_train.append(DataTrigramVec(tmp))
    
#print(X_train)

In [24]:
print(type(X_train))

<class 'list'>


In [25]:
#строка триграмм в случае с верхним регистром получается слишком длинной, с этим невозможно было работать
messages_num = len(X_train)
cols_num = len(X_train[0])

print('колличество признаков: ', cols_num, '\n', 'колличество сообщений: ', messages_num) 

колличество признаков:  21877 
 колличество сообщений:  3900


In [26]:
#cv = KFold(n_splits = 5, shuffle = True, random_state = 777)
#clf = SVC(kernel = 'linear', random_state = 777)

#gs = GridSearchCV(clf, grid, scoring = 'accuracy', cv = cv)
#gs.fit(X_train, y_train)

#clf = SVC(kernel = 'linear', random_state = 777, C = gs.best_params_['C'])
#clf.fit(X_train, y_train)

trig_clf = SVC(kernel = 'linear', random_state = 777, C = 1)
trig_clf.fit(X_train, y_train)

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=777,
    shrinking=True, tol=0.001, verbose=False)

In [21]:
X_test = np.array(x_test)
test_x_pred_vect = []
for tmp in X_test:
    test_x_pred_vect.append(DataTrigramVec(tmp))
    
data_y_pred = trig_clf.predict(test_x_pred_vect)

for i in data_y_pred[-10:]:
    print(i)

0
0
0
0
1
1
0
0
0
0


In [22]:
X_test[-5] #пятое сообщение с конца помечено 1 - то есть является спамом

'This is the 2nd time we have tried 2 contact u. U have won the еЈ750 Pound prize. 2 claim is easy, call 087187272008 NOW1! Only 10p per minute. BT-national-rate.'

In [23]:
#Сравнение разных метрик
#в данном случае negatine = ham, positive = spam

test_len = len(y_test)
pos_num = sum(y_test)
neg_num = test_len - pos_num

true_pos = 0
true_neg = 0
for i in range(test_len):
    if data_y_pred[i] == y_test[i] == 1:
        true_pos += 1
    if data_y_pred[i] == y_test[i] == 0:
        true_neg += 1

false_pos = pos_num - true_pos
false_neg = neg_num - true_neg

print('accuracy =', (true_pos + true_neg) / test_len)
print('precision = ', true_pos / (true_pos + false_pos))
print('recall = ', true_pos / (true_pos + false_neg))

accuracy = 0.9784688995215312
precision =  0.8771929824561403
recall =  0.9615384615384616


In [39]:
trig_clf.coef_[0]

array([ 0.        , -0.04746957,  0.        , ...,  0.        ,
        0.        , -0.01145405])

In [24]:
#Наиболее значимые триграммы для выявления неспама

A = trig_clf.coef_[0]
trig_imp_ham = []

for j in range(cols_num):
    
    min_coef = 0
    min_coef_ind = 0
    
    for i in range(len(A)):
        if  A[i] < min_coef:
            min_coef = A[i]
            min_coef_ind = i
    
    A = np.delete(A, min_coef_ind)
    trig_imp_ham.append(trigram_vec[min_coef_ind])
    #print(trigram_vec[min_coef_ind])
    
print(trig_imp_ham[:30])

['lly', 'reo', 'llz', 'ls.', 'i c', ' go', 'lry', 'ful', 'hhz', 'yst', 'es.', 'yhs', 'ilg', '..n', 'umr', 'ee ', 'ojq', 'ang', 'wdo', 'lhu', ' al', 'uem', 'aty', 'hhu', 'cex', 'khq', 'rqi', 'hee', 'eek', 'fdu']


In [40]:
#А это наиболее значимые триграммы непосредственно для выявления спама

A = trig_clf.coef_[0]
trig_imp_spam = []

for j in range(cols_num):
    
    max_coef = 0
    max_coef_ind = 0
    
    for i in range(len(A)):
        if  A[i] > max_coef:
            max_coef = A[i]
            max_coef_ind = i
    
    A = np.delete(A, max_coef_ind)
    trig_imp_spam.append(trigram_vec[max_coef_ind])
    #print(trigram_vec[max_coef_ind])
    
print(trig_imp_spam[:30])

['nfo', 'inf', 'sia', 'rec', ' pm', 'epl', 'plv', 'xsw', 'ugv', 'osq', ' ly', 'z.r', 'all', 'nor', 'lk.', 'thm', 'sem', 'v.x', 'imz', 'neq', 'anm', 'grz', 'tnl', 'ett', 'sdl', '.bt', 'mnw', 'exu', 'tw.', 'aim']


In [26]:
tmp = np.where(np.array(trigram_vec) == 'com')[0][0]

print(tmp) # nfo находится на 10697 месте

2055


In [38]:
# узнаем веса интересующих нас триграмм в модели

for trig in ['lly', '???', 'www', 'nfo', 'com', 'you', 'all', '.co', '.uk', 'amb', 'txt', 'epl']:
    print(f'вес триграммы {trig}:', trig_clf.coef_[0][np.where(np.array(trigram_vec) == trig)[0][0]])


вес триграммы lly: -0.3423899270953178
вес триграммы ???: -0.011454052473612265
вес триграммы www: 0.12268766564072589
вес триграммы nfo: 0.7934107666304415
вес триграммы com: -0.06750669239546953
вес триграммы you: -0.16930482229793237
вес триграммы all: 0.34754825864086547
вес триграммы .co: 0.24404056238528804
вес триграммы .uk: 0.1794366528178536
вес триграммы amb: -0.04108883317870469
вес триграммы txt: 0.2398368855887745
вес триграммы epl: 0.5387869721363082


In [28]:
new_df = pd.read_excel('spam_testing.xlsx')
new = []

for str_ in new_df['Spam']:
    new.append(str_)
    
new_pred_vect = []
for tmp in new:
    new_pred_vect.append(DataTrigramVec(tmp))
new_pred = trig_clf.predict(new_pred_vect)

for i in range(len(new)):
    print(new[i], ' - ', (lambda x: x == 1 and 'spam' or 'ham')(new_pred[i]), '\n')


We shall await your immediate correspondence with the information requested above to his private email address at (raphaelbostic@usa.com) for fast re-confirmation so that he may conclude your payment immediately.  -  spam 

You are everywhere dirt, on the floor, the windows, even on my shirt. And sometimes when i open my mouth, you are all that comes flowing out. I dream of my world without you, then half my chores are out too. A time of joy for me, lots of tv shows i.ll see. But i guess like all things you just must exist, like rain, hail and mist, and when my time here is done, you and i become one.  -  ham 

We are pleased to inform you that an amount of £500,000.00 (GBP) has been donated and given, gifted to you and your family by Steve & Lenka Thomson, who won the EuroMillions jackpot, lottery of £105,100,701.90 Euro Millions, part of this donation,       it is for you and your family. This donation is to help fight against CoronaVirus COVID-19 pandemic in the world, and help the 

In [44]:
#старый вариант ввода на триграммах

new = np.array(['free info get it all', 'Also Do Knock-Knock Jokes. For example: -knock-knock -who is that? -Ella -Ella Who? -Ella Who Akbar...',\
                'hey winner get your prize. all info in our site www.notspam.bs', 'hi buddy how you doin?', 'hey you are a winner get your prize, call us. all info at the discription'])

new_pred_vect = []
for tmp in new:
    new_pred_vect.append(DataTrigramVec(tmp))
new_pred = trig_clf.predict(new_pred_vect)

for i in range(len(new)):
    print(new[i], ' - ', (lambda x: x == 1 and 'spam' or 'ham')(new_pred[i]), '\n')


free info get it all  -  spam 

Also Do Knock-Knock Jokes. For example: -knock-knock -who is that? -Ella -Ella Who? -Ella Who Akbar...  -  ham 

hey winner get your prize. all info in our site www.notspam.bs  -  spam 

hi buddy how you doin?  -  ham 

hey you are a winner get your prize, call us. all info at the discription  -  spam 



In [46]:
#старый вариант ввода на tfidf

new = np.array(['free info get it all', 'Also Do Knock-Knock Jokes. For example: -knock-knock -who is that? -Ella -Ella Who? -Ella Who Akbar...',\
                'hey winner get your prize. all info in our site www.notspam.bs', 'hi buddy how you doin?', 'hey you are a winner get your prize'])

new_pred_vect = vectorizer.transform(new)
new_pred = clf.predict(new_pred_vect)

for i in range(len(new)):
    print(new[i], ' - ', (lambda x: x == 1 and 'spam' or 'ham')(new_pred[i]), '\n')


free info get it all  -  ham 

Also Do Knock-Knock Jokes. For example: -knock-knock -who is that? -Ella -Ella Who? -Ella Who Akbar...  -  ham 

hey winner get your prize. all info in our site www.notspam.bs  -  spam 

hi buddy how you doin?  -  ham 

hey you are a winner get your prize  -  spam 

